In [10]:
import cv2
import math
import numpy as np
import sys

def apply_mask(matrix, mask, fill_value):
    #print("MATRIX=", matrix)
    #print("mask=\n" ,mask)
    #print("fill value=\n", fill_value)
                 
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    print('MASKED=',masked)
    return masked.filled()

def apply_threshold(matrix, low_value=255, high_value=255):
    low_mask = matrix < low_value
    print("low mask=",low_mask)
    
    matrix = apply_mask(matrix, low_mask, low_value)
    print('Low MASK->',low_mask,'\nMatrix->',matrix)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100
    print("shape of image = ", img.shape[2])

    half_percent = percent / 200.0
    print('HALF PERCENT->',half_percent)

    channels = cv2.split(img)
    print('Channels->\n',channels)
    print('Shape->',channels[0].shape)
    print('Shape of channels->',len(channels[2]))

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2

	# find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)
        print('vec=',vec_size,'\nFlat=',flat)
        assert len(flat.shape) == 1

        flat = np.sort(flat)

        n_cols = flat.shape[0]
        print("Number of columns = ", n_cols)

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        print("Lowval: ", low_val)
        print("Highval: ", high_val)
        print(flat[60])
        print(flat[11940])
        

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)

if __name__ == '__main__':
    #img = cv2.imread(sys.argv[1])
    cap=cv2.VideoCapture('/home/dheeraj/Downloads/Barracuda.mp4')
    #img = cv2.imread('/home/dheeraj/Downloads/Whale.mov')
    #cap.set(cv2.cv.CV_CAP_PROP_FPS, 60)
    
    while True:
        ret, frame=cap.read()
        frame_width_fx = 640
        frame_height_fy = 480
        frame = cv2.resize(frame, (frame_width_fx, frame_height_fy))
        

        out = simplest_cb(frame, 1)
        cv2.imshow("Before", frame) 
        cv2.imshow("After", out)
        key=cv2.waitKey(1)
        if key==27:
            break
        
    cap.release()
    cv2.destroyAllWindows()


shape of image =  3
HALF PERCENT-> 0.005
Channels->
 [array([[100, 154, 156, ..., 161, 164,  79],
       [151, 226, 228, ..., 234, 238, 137],
       [158, 234, 236, ..., 235, 239, 137],
       ...,
       [ 91, 132, 140, ..., 154, 155, 101],
       [ 95, 130, 132, ..., 144, 160, 100],
       [ 72, 101, 100, ...,  98, 112,  68]], dtype=uint8), array([[ 82, 135, 134, ...,  98,  98,  53],
       [117, 194, 192, ..., 145, 146,  84],
       [118, 196, 193, ..., 145, 146,  84],
       ...,
       [ 70, 100, 100, ..., 129, 133,  86],
       [ 83, 109, 101, ..., 126, 145,  90],
       [ 70,  89,  78, ...,  87, 103,  66]], dtype=uint8), array([[ 48,  61,  64, ...,  20,  18,  21],
       [ 68, 102, 102, ...,  32,  29,  18],
       [ 63,  96, 100, ...,  32,  29,  18],
       ...,
       [ 49,  67,  66, ...,  90,  93,  50],
       [ 61,  74,  68, ...,  88, 107,  58],
       [ 46,  57,  48, ...,  50,  68,  37]], dtype=uint8)]
Shape-> (480, 640)
Shape of channels-> 480
vec= 307200 
Flat= [100 154 15

Channels->
 [array([[100, 154, 156, ..., 161, 164,  79],
       [151, 226, 228, ..., 234, 238, 137],
       [158, 234, 236, ..., 235, 239, 137],
       ...,
       [ 98, 144, 170, ..., 154, 155, 101],
       [ 91, 135, 146, ..., 144, 160, 100],
       [ 67,  95,  99, ...,  98, 112,  68]], dtype=uint8), array([[ 82, 135, 134, ...,  98,  98,  53],
       [117, 194, 192, ..., 145, 146,  84],
       [118, 196, 193, ..., 145, 146,  84],
       ...,
       [ 58,  93, 112, ..., 129, 133,  86],
       [ 60,  97, 102, ..., 126, 145,  90],
       [ 41,  65,  61, ...,  87, 103,  66]], dtype=uint8), array([[ 48,  61,  64, ...,  20,  18,  21],
       [ 68, 102, 102, ...,  32,  29,  18],
       [ 63,  96, 100, ...,  32,  29,  18],
       ...,
       [ 36,  62,  71, ...,  90,  93,  50],
       [ 36,  65,  64, ...,  88, 107,  58],
       [ 21,  38,  29, ...,  50,  68,  37]], dtype=uint8)]
Shape-> (480, 640)
Shape of channels-> 480
vec= 307200 
Flat= [100 154 156 ...  98 112  68]
Number of columns =  3

 [85 85 93 ... 88 95 85]]
vec= 307200 
Flat= [48 61 64 ... 52 60 37]
Number of columns =  307200
Lowval:  0
Highval:  172
0
2
low mask= [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
MASKED= [[48 61 64 ... 17 15 16]
 [68 102 102 ... 28 25 10]
 [63 96 100 ... 27 24 10]
 ...
 [45 81 79 ... 87 93 47]
 [42 74 83 ... 88 99 57]
 [22 42 49 ... 52 60 37]]
Low MASK-> [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]] 
Matrix-> [[ 48  61  64 ...  17  15  16]
 [ 68 102 102 ...  28  25  10]
 [ 63  96 100 ...  27  24  10]
 ...
 [ 45  81  79 ...  87  93  47]
 [ 42  74  83 ...  

Channels->
 [array([[100, 154, 156, ..., 159, 159,  77],
       [151, 226, 228, ..., 233, 232, 132],
       [158, 234, 236, ..., 233, 233, 133],
       ...,
       [120, 186, 179, ..., 158, 168,  98],
       [110, 167, 174, ..., 148, 154, 100],
       [ 77, 125, 125, ..., 105, 107,  72]], dtype=uint8), array([[ 82, 135, 134, ...,  98,  96,  52],
       [117, 194, 192, ..., 146, 142,  81],
       [118, 196, 193, ..., 146, 142,  81],
       ...,
       [ 96, 154, 151, ..., 125, 137,  75],
       [ 91, 142, 150, ..., 121, 129,  80],
       [ 60, 103, 111, ...,  84,  88,  60]], dtype=uint8), array([[ 48,  61,  64, ...,  17,  15,  16],
       [ 68, 102, 102, ...,  28,  25,  10],
       [ 63,  96, 100, ...,  27,  24,  10],
       ...,
       [ 60,  89,  78, ...,  86,  98,  41],
       [ 56,  85,  86, ...,  85,  93,  50],
       [ 29,  57,  62, ...,  49,  55,  33]], dtype=uint8)]
Shape-> (480, 640)
Shape of channels-> 480
vec= 307200 
Flat= [100 154 156 ... 105 107  72]
Number of columns =  3

 [123 123 123 ... 123 123 123]]
vec= 307200 
Flat= [ 82 135 134 ...  87  98  48]
Number of columns =  307200
Lowval:  86
Highval:  246
71
121
low mask= [[ True False False ... False False  True]
 [False False False ... False False  True]
 [False False False ... False False  True]
 ...
 [False False False ... False False  True]
 [False False False ... False False  True]
 [ True False False ... False False  True]]
MASKED= [[-- 135 134 ... 96 93 --]
 [117 194 192 ... 142 140 --]
 [118 196 193 ... 142 139 --]
 ...
 [93 151 137 ... 138 143 --]
 [87 146 135 ... 132 141 --]
 [-- 94 94 ... 87 98 --]]
Low MASK-> [[ True False False ... False False  True]
 [False False False ... False False  True]
 [False False False ... False False  True]
 ...
 [False False False ... False False  True]
 [False False False ... False False  True]
 [ True False False ... False False  True]] 
Matrix-> [[ 86 135 134 ...  96  93  86]
 [117 194 192 ... 142 140  86]
 [118 196 193 ... 142 139  86]
 ...
 [ 93 151 137 ...

...

Number of columns =  307200
Lowval:  82
Highval:  244
61
123
low mask= [[ True False False ... False False  True]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True False ... False False False]
 [ True False False ... False False False]
 [ True  True  True ... False False False]]
MASKED= [[-- 125 125 ... 96 96 --]
 [114 187 187 ... 141 142 86]
 [113 186 188 ... 142 142 83]
 ...
 [-- -- 85 ... 218 220 139]
 [-- 83 82 ... 211 219 139]
 [-- -- -- ... 151 152 97]]
Low MASK-> [[ True False False ... False False  True]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True False ... False False False]
 [ True False False ... False False False]
 [ True  True  True ... False False False]] 
Matrix-> [[ 82 125 125 ...  96  96  82]
 [114 187 187 ... 141 142  86]
 [113 186 188 ... 142 142  83]
 ...
 [ 82  82  85 ... 218 220 139]
 [ 82  83  82 ... 211 219 139]
 [ 82  82  82 ... 151 152  97]]
MASK

MASKED= [[40 63 64 ... 20 17 16]
 [59 94 93 ... 24 25 17]
 [62 93 89 ... 23 23 18]
 ...
 [29 55 55 ... 111 117 82]
 [29 55 58 ... 109 123 81]
 [21 35 36 ... 77 85 53]]
Low MASK-> [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]] 
Matrix-> [[ 40  63  64 ...  20  17  16]
 [ 59  94  93 ...  24  25  17]
 [ 62  93  89 ...  23  23  18]
 ...
 [ 29  55  55 ... 111 117  82]
 [ 29  55  58 ... 109 123  81]
 [ 21  35  36 ...  77  85  53]]
MASKED= [[40 63 64 ... 20 17 16]
 [59 94 93 ... 24 25 17]
 [62 93 89 ... 23 23 18]
 ...
 [29 55 55 ... 111 117 82]
 [29 55 58 ... 109 123 81]
 [21 35 36 ... 77 85 53]]
shape of image =  3
HALF PERCENT-> 0.005
Channels->
 [array([[ 96, 161, 157, ..., 150, 154,  88],
       [143, 231, 232, ..., 233, 234, 136],
       [143, 234, 233, ..., 234, 234, 140],
 

Number of columns =  307200
Lowval:  123
Highval:  242
84
197
low mask= [[ True False False ... False False  True]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True False ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False  True]]
MASKED= [[-- 161 157 ... 150 154 --]
 [143 231 232 ... 235 236 138]
 [143 234 233 ... 234 234 140]
 ...
 [-- -- 124 ... 240 236 148]
 [-- -- -- ... 236 234 144]
 [-- -- -- ... 153 154 --]]
Low MASK-> [[ True False False ... False False  True]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True False ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False  True]] 
Matrix-> [[123 161 157 ... 150 154 123]
 [143 231 232 ... 235 236 138]
 [143 234 233 ... 234 234 140]
 ...
 [123 123 124 ... 240 236 148]
 [123 123 123 ... 236 234 144]
 [123 123 123 ... 153 154 123]

MASKED= [[82 124 123 ... 94 94 82]
 [113 186 185 ... 139 140 86]
 [110 184 186 ... 139 139 82]
 ...
 [82 82 82 ... 222 200 88]
 [82 82 82 ... 208 183 82]
 [82 82 82 ... 137 117 82]]
vec= 307200 
Flat= [40 62 63 ... 67 49 29]
Number of columns =  307200
Lowval:  0
Highval:  164
0
1
low mask= [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
MASKED= [[40 62 63 ... 18 16 16]
 [59 93 93 ... 23 24 17]
 [61 92 89 ... 21 21 16]
 ...
 [39 58 56 ... 116 99 35]
 [45 56 55 ... 102 80 36]
 [30 38 32 ... 67 49 29]]
Low MASK-> [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]] 
Ma

shape of image =  3
HALF PERCENT-> 0.005
Channels->
 [array([[ 96, 160, 156, ..., 151, 156,  91],
       [143, 230, 232, ..., 235, 235, 137],
       [143, 235, 235, ..., 234, 234, 140],
       ...,
       [ 66, 114, 115, ..., 246, 221, 106],
       [ 66, 111, 116, ..., 236, 212,  98],
       [ 42,  74,  78, ..., 148, 135,  64]], dtype=uint8), array([[ 75, 124, 123, ...,  94,  96,  56],
       [113, 186, 185, ..., 139, 139,  85],
       [111, 186, 188, ..., 138, 138,  81],
       ...,
       [ 50,  82,  83, ..., 188, 145,  80],
       [ 51,  81,  81, ..., 164, 138,  74],
       [ 34,  53,  53, ..., 108,  94,  49]], dtype=uint8), array([[40, 62, 63, ..., 19, 17, 17],
       [59, 93, 93, ..., 23, 23, 16],
       [62, 94, 91, ..., 20, 20, 16],
       ...,
       [32, 60, 61, ..., 81, 40, 51],
       [33, 59, 60, ..., 65, 52, 52],
       [24, 39, 37, ..., 37, 38, 31]], dtype=uint8)]
Shape-> (480, 640)
Shape of channels-> 480
vec= 307200 
Flat= [ 96 160 156 ... 148 135  64]
Number of columns